In [21]:
import cv2
import numpy as np
from collections import deque
import time


In [ ]:
class LivenessDetector:
    def __init__(self):
        # Load face cascade
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
                
        # More strict thresholds
        self.texture_threshold = 25  # Increased for better fake detection
        self.motion_history = deque(maxlen=30)
        self.consecutive_closed = 0
        
        # Color analysis history
        self.color_variations = deque(maxlen=30)
        
    def calculate_texture_score(self, face_roi):
        """Calculate texture complexity - real skin has more variation"""
        gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
        
        # Multiple texture measures
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
        
        # Local binary pattern-like measure
        sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
        edge_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
        edge_score = edge_magnitude.std()
        
        # Combined score
        texture_score = laplacian_var + edge_score
        return texture_score
    
    def detect_motion(self, face_coords):
        """Detect natural micro-movements with stricter requirements"""
        if len(self.motion_history) < 10:
            self.motion_history.append(face_coords)
            return 0, False
        
        self.motion_history.append(face_coords)
        
        # Calculate movement variance
        positions = np.array(list(self.motion_history))
        variance = np.var(positions, axis=0).sum()
        
        # Check for natural movement pattern (not too still, not too erratic)
        is_natural = 2 < variance < 80
        
        return variance, is_natural
    
    def analyze_color_variation(self, face_roi):
        """Real faces have color variation due to blood flow"""
        # Convert to different color space
        hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)
        ycrcb = cv2.cvtColor(face_roi, cv2.COLOR_BGR2YCrCb)
        
        # Calculate color channel variations
        h_var = np.var(hsv[:,:,0])
        cr_var = np.var(ycrcb[:,:,1])
        
        color_score = h_var + cr_var
        self.color_variations.append(color_score)
        
        # Real faces should have consistent color variation
        if len(self.color_variations) > 20:
            variation_consistency = np.std(list(self.color_variations))
            return color_score, variation_consistency < 50
        
        return color_score, False
 
    def check_screen_reflection(self, face_roi):
        """Detect screen reflections and uniform lighting typical of fake images"""
        gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
        
        # Check for uniform brightness (screens often have this)
        brightness_std = np.std(gray)
        
        # Check for regular patterns (screen pixels)
        fft = np.fft.fft2(gray)
        fft_shift = np.fft.fftshift(fft)
        magnitude_spectrum = np.abs(fft_shift)
        
        # Screens often have periodic patterns
        spectrum_peaks = np.sort(magnitude_spectrum.flatten())[-100:].mean()
        
        # Low brightness variation + high spectrum peaks = likely screen
        is_screen_like = brightness_std < 30 and spectrum_peaks > 10000
        
        return brightness_std, is_screen_like
    
    def is_real_face(self, frame, face_coords):
        """Determine if face is real with stricter criteria"""
        x, y, w, h = face_coords
        face_roi = frame[y:y+h, x:x+w]
        gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
        
        # Test 1: Texture analysis (stricter)
        texture_score = self.calculate_texture_score(face_roi)
        texture_passed = texture_score > self.texture_threshold
        
        # Test 2: Motion analysis (stricter)
        motion_score, motion_natural = self.detect_motion((x, y))
                
        # Test 3: Color variation analysis
        color_score, color_natural = self.analyze_color_variation(face_roi)
        
        # Test 4: Screen reflection detection
        brightness_std, is_screen = self.check_screen_reflection(face_roi)
        screen_passed = not is_screen
        
        # Stricter scoring system
        score = 0
        confidence_factors = []
        
        if texture_passed:
            score += 2  # Texture is very important
            confidence_factors.append("texture")
        
        if motion_natural:
            score += 2  # Natural motion is crucial
            confidence_factors.append("motion")
        
        if color_natural:
            score += 1
            confidence_factors.append("color")
        
        if screen_passed:
            score += 1
            confidence_factors.append("no_screen")
        else:
            score -= 2  # Penalize screen-like characteristics
        
        # Need at least 5 points to be considered real (stricter)
        is_real = score >= 5
        
        return is_real, {
            'texture_score': texture_score,
            'texture_passed': texture_passed,
            'motion_score': motion_score,
            'motion_natural': motion_natural,
            'color_score': color_score,
            'brightness_std': brightness_std,
            'is_screen_like': is_screen,
            'total_score': score,
            'max_score': 9,
            'confidence_factors': confidence_factors
        }


In [23]:
def main(duration=10):
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    detector = LivenessDetector()
    
    print("Face Liveness Detection Started")    
    # Timer settings
    start_time = time.time()
    
    # Store results for final analysis
    all_results = []
    face_detected_frames = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Calculate remaining time
        elapsed = time.time() - start_time
        remaining = max(0, duration - elapsed)
        
        # Check if time is up
        if remaining == 0:
            break
        
        # Flip frame for mirror effect
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces
        faces = detector.face_cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100)
        )
        
        if len(faces) > 0:
            face_detected_frames += 1
        
        for (x, y, w, h) in faces:
            # Determine if real or fake
            is_real, metrics = detector.is_real_face(frame, (x, y, w, h))
            
            # Store results
            all_results.append({
                'is_real': is_real,
                'metrics': metrics
            })
            
            # Draw rectangle and label
            color = (0, 255, 0) if is_real else (0, 0, 255)
            label = "REAL" if is_real else "FAKE/SPOOF"
            
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
            cv2.putText(frame, label, (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
            
            # Display live metrics
            y_offset = y + h + 25
            line_height = 18
            
            cv2.putText(frame, f"Texture: {metrics['texture_score']:.1f} {'✓' if metrics['texture_passed'] else '✗'}", 
                       (x, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
            cv2.putText(frame, f"Motion: {metrics['motion_score']:.1f} {'✓' if metrics['motion_natural'] else '✗'}", 
                       (x, y_offset + line_height), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
            cv2.putText(frame, f"Screen: {'Yes ✗' if metrics['is_screen_like'] else 'No ✓'}", 
                       (x, y_offset + line_height*4), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
            cv2.putText(frame, f"Score: {metrics['total_score']}/{metrics['max_score']}", 
                       (x, y_offset + line_height*5), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 255), 1)
        
        # Display countdown timer (large and prominent)
        countdown_text = f"{int(remaining)}"
        cv2.putText(frame, countdown_text, (frame.shape[1]//2 - 30, 80), 
                   cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0, 255, 255), 4)
        
        # Display instructions
        cv2.putText(frame, "Smile, life is beautiful :) ", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow('Face Liveness Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

    # Analyze results
    if all_results:
        print("\n" + "="*60)
        print("LIVENESS DETECTION RESULT")
        print("="*60)
        
        # Calculate statistics
        real_count = sum(1 for r in all_results if r['is_real'])
        total_count = len(all_results)
        
        avg_texture = np.mean([r['metrics']['texture_score'] for r in all_results])
        avg_motion = np.mean([r['metrics']['motion_score'] for r in all_results])
        avg_brightness = np.mean([r['metrics']['brightness_std'] for r in all_results])
        screen_detections = sum(1 for r in all_results if r['metrics']['is_screen_like'])
        
        # Evaluation criteria
        texture_ok = avg_texture > detector.texture_threshold
        motion_ok = 2 < avg_motion < 80
        screen_ok = screen_detections < (total_count * 0.3)
        
        criteria_passed = sum([texture_ok, motion_ok, screen_ok])
        probability = (real_count / total_count) * 100
        
        final_verdict = "REAL" if criteria_passed >= 2 and probability >= 60 else "FAKE"
        confidence = min(probability, 95) if final_verdict == "REAL" else max(100 - probability, 60)
        
        print(f"\nVerdict: {final_verdict}")
        print(f"Confidence: {confidence:.1f}%\n")
        
        print(f"Texture: {avg_texture:.1f} (need >{detector.texture_threshold}) {'✓' if texture_ok else '✗'}")
        print(f"Motion: {avg_motion:.1f} (need 2-80) {'✓' if motion_ok else '✗'}")
        print(f"Screen-like: {screen_detections}/{total_count} frames {'✓' if screen_ok else '✗'}")
        
        print(f"\nReason: ", end="")
        if final_verdict == "REAL":
            reasons = []
            if texture_ok: reasons.append("natural texture")
            if motion_ok: reasons.append("natural motion")
            if screen_ok: reasons.append("no screen artifacts")
            print(", ".join(reasons))
        else:
            reasons = []
            if not texture_ok: reasons.append("flat texture")
            if not motion_ok: reasons.append("unnatural motion")
            if not screen_ok: reasons.append("screen detected")
            print(", ".join(reasons))
        
        print("="*60)
    else:
        print("\n❌ No face detected!")


In [24]:

if __name__ == "__main__":
    main()

Face Liveness Detection Started

LIVENESS DETECTION RESULT

Verdict: FAKE
Confidence: 100.0%

Texture: 508.5 (need >25) ✓
Motion: 6246.7 (need 2-80) ✗
Screen-like: 358/455 frames ✗

Reason: unnatural motion, screen detected
